In [20]:
# ==========================================
# 各種ライブラリと CSV データの読み込み
# ==========================================
# 自動リロードを有効にする設定
%load_ext autoreload
%autoreload 2

# 必要なライブラリのインストール
%pip install pycountry

# ライブラリのインポート
import numpy as np  # 線形代数
import pandas as pd  # データ処理、CSVファイルのI/O（例：pd.read_csv）
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

# 独自モジュールのインポート
from modules.normalizer import TextNormalizer, AdditionalNormalizer

# CSVデータを pandas データフレームオブジェクトとして読み込み
df = pd.read_csv("../input/nlp-getting-started/train.csv")
df = df.fillna("")  # 空のカラムを空文字に置換

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [21]:
# ==========================================
# "text" カラムの前処理
# ==========================================
X_text = df[["text"]].copy()
# X_text = X_text[51:101].copy()  # テスト用: 50~100 だけ抽出

X_text["text"] = (
    X_text["text"]
    .apply(TextNormalizer.remove_newlines)
    .apply(TextNormalizer.replace_links)
)

In [22]:
# ==========================================
# "keyword" カラムの前処理
# ==========================================
X_keyword = df[["keyword"]].copy()

In [23]:
# ==========================================
# "location" カラムの前処理
# ==========================================
X_location = df[["location"]].copy()
# X_location = X_location[51:101].copy()  # テスト用: 50~100 だけ抽出

X_location["location"] = (
    X_location["location"]
    .apply(TextNormalizer.remove_numbers_and_symbols)
    .apply(AdditionalNormalizer.normalize_country_name)
)

In [24]:
# ==========================================
# モデルの学習と評価
# ==========================================

# DataFrame として結合 (axis=1 で横方向に結合)
X_keyword_and_location = pd.concat([X_keyword, X_location], axis=1)

# get_dummies で One-Hot エンコーディング
X_keyword_and_location_dummies = pd.get_dummies(
    X_keyword_and_location, columns=["keyword", "location"], dummy_na=True
)  # dummy_na=True で NaN を特徴量化

# "text" カラムのテキストデータをTF-IDFベクトルに変換
vectorizer = TfidfVectorizer(max_features=1000)
X_text_tfidf = vectorizer.fit_transform(X_text["text"]).toarray()

# TF-IDF 特徴量と結合 (こちらは numpy array 同士なので hstack でOK)
X = np.hstack((X_keyword_and_location_dummies.values, X_text_tfidf))
y = df["target"]

# データを訓練セットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 学習されていないモデルオブジェクト
clf = RandomForestClassifier(random_state=42)

# モデルの訓練
clf.fit(X_train, y_train)

# テストセットで予測
y_pred = clf.predict(X_test)

# 精度の計算
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.79


In [15]:
# test.csv に対して予測を行う
df_prod = pd.read_csv("../input/nlp-getting-started/test.csv")
X_prod_text_tfidf = vectorizer.transform(df_prod["text"]).toarray()

# 訓練データと同じダミー変数構造を作成するため、元の訓練データの列名を取得
X_prod = df_prod[["keyword", "location"]].copy()
X_prod_dummies = pd.get_dummies(X_prod, columns=["keyword", "location"], dummy_na=True)

# 訓練データで使用した特徴量の列名を取得 (TF-IDF部分を除く)
train_feature_cols = pd.get_dummies(
    df[["keyword", "location"]], columns=["keyword", "location"], dummy_na=True
).columns

# 訓練データに存在してテストデータに存在しない列を特定
missing_cols = set(train_feature_cols) - set(X_prod_dummies.columns)

# 欠落した列をすべて一度に追加 (パフォーマンス警告を回避)
if missing_cols:
    missing_df = pd.DataFrame(0, index=X_prod_dummies.index, columns=list(missing_cols))
    X_prod_dummies = pd.concat([X_prod_dummies, missing_df], axis=1)

# テストデータに存在して訓練データに存在しない列を削除
X_prod_dummies = X_prod_dummies[train_feature_cols]

# TF-IDF特徴量と結合
X_prod = np.hstack((X_prod_dummies.values, X_prod_text_tfidf))
y_pred_prod = clf.predict(X_prod)

# 提出用ファイルの作成
submission = pd.DataFrame({"id": df_prod["id"], "target": y_pred_prod})
submission.to_csv("../output/submission.csv", index=False)

ValueError: X has 4564 features, but RandomForestClassifier is expecting 1409 features as input.